In [1]:
#Task: ABSA
#Language: Arabic
#Contact: Lily Foula <lilyfoula@gmail.com>
#Last Update: 2025-06-26
#Requirements: flair, transformers, ipymarkup
#Encoding: UTF-8

# Aspect-Based Sentiment Analysis (ABSA) with FLAIR

This Notebook shows you how to perform ABSA, which consists of two sub-tasks.
- Task A: Extraction of **Aspects** from unlabelled text
- Task B: Finding the sentiment associated with each **Aspect**.

**Aspects** can be any entities of interest in the text for which we want to find out the associated sentiment. This mostly depends on the kind of data you are working on. For example, the **Aspects** in a news corpora might be person names and organizations, while **Aspects** in a dataset of restaurant reviews might be names of dishes and drinks.

For this reason, we recommend training your own AE (Task A: Aspect Extraction) system using this notebook as a guide. For this task you, will need an annotated dataset. The sample dataset used here deals with hotel reviews and includes various annotated aspects like Hotel, Service, etc. 

❗ To understand and adapt this Notebook for your own use case, we expect a basic understanding of these concepts:

- The task of ABSA
- Fine-tuning
- Language Models
- Python and frequently used libraries (HuggingFace Transformers, FLAIR, scikit-learn, etc)



#### Installing the Dependencies

We will first install all the libraries needed to run this notebook, this might take a few minutes.

In [2]:
!pip install flair transformers torch pandas matplotlib ipymarkup scikit-learn statistics tqdm

## Section 1: Fine-tuning a Model with your annotated dataset

In this section, we will cover how you can fine-tune models for the task of ABSA from scratch using a few tools like transformers, the FLAIR-NLP toolkit, etc.
As detailed above, ABSA consists of two tasks Aspect Extraction (Task A) and Sentiment Classification (Task B), let's look into Task A first.

### Task A: Aspect Extraction

The first and hardest task of ABSA, involves finding interesting aspects in unstructured text. The aspects can be a single word or a large phrase or named entity. This makes the task more difficult since there is no standard word length or limits for the aspects.  

#### A.1 Loading and checking your Data

Now we will load our annotated dataset and do some quick checks to see how it looks. We have fine-tuned on our sample dataset. 
For this task, we need our dataset to be in the **.txt format**. Our original dataset was in **.xml format**. That is why we added the following two code cells to transform it to **.txt format**. The third code cell cleans our dataset to get best results. 

To train on your data, replace the paths of the files with your personal created dataset. Check what format your dataset is and based on that, follow the instructions in the following code cells to know which cells to run and which to ignore. Otherwise, you might run into issues!

In [3]:
# Start from this code cell and run all of the following if your dataset is in .xml format
import xml.etree.ElementTree as ET
import re

def tokenize(text):
    # Simple tokenizer: splits words and punctuation
    return re.findall(r'\w+|[^\w\s]', text)

def convert_semeval_to_bio(xml_path, output_path):
    tree = ET.parse(xml_path)
    root = tree.getroot()

    with open(output_path, "w", encoding="utf-8") as out_file:
        for review in root.findall("Review"):
            for sentence in review.findall(".//sentence"):
                text_element = sentence.find("text")
                if text_element is None or text_element.text is None:
                    continue

                text = text_element.text.strip()
                tokens = tokenize(text)
                labels = ["O"] * len(tokens)

                # Character offset to token mapping
                char_to_token_idx = []
                char_index = 0
                for token in tokens:
                    start_idx = text.find(token, char_index)
                    end_idx = start_idx + len(token)
                    char_to_token_idx.append((start_idx, end_idx))
                    char_index = end_idx

                opinions = sentence.find("Opinions")
                if opinions is not None:
                    for opinion in opinions.findall("Opinion"):
                        target = opinion.attrib["target"]
                        if target == "NULL":
                            continue  # Skip implicit aspect targets

                        category = opinion.attrib["category"]
                        base_category = category.split("#")[0]  # e.g., "ROOMS_AMENITIES"

                        start = int(opinion.attrib["from"])
                        end = int(opinion.attrib["to"])

                        # Find matching tokens
                        target_token_indexes = []
                        for i, (char_start, char_end) in enumerate(char_to_token_idx):
                            if char_start >= start and char_end <= end:
                                target_token_indexes.append(i)

                        if not target_token_indexes:
                            continue

                        # Tag tokens using BIO format with category
                        labels[target_token_indexes[0]] = f"B-{base_category}"
                        for i in target_token_indexes[1:]:
                            labels[i] = f"I-{base_category}"

                # Write sentence in CoNLL format
                for token, label in zip(tokens, labels):
                    out_file.write(f"{token}\t{label}\n")
                out_file.write("\n")  # Sentence boundary


In [4]:
convert_semeval_to_bio("AR_Hotels_Train_SB1.xml", "iob_output_new.txt")

In [5]:
# This code cell cleans our dataset for better training results
input_file = "iob_output_new.txt"
output_file = "iob_output_relabelled.txt"

# Define labels you want to remove
labels_to_remove = {"B-ROOMS_AMENITIES", "I-ROOMS_AMENITIES", "B-FACILITIES", "I-FACILITIES"}

with open(input_file, "r", encoding="utf-8") as infile, open(output_file, "w", encoding="utf-8") as outfile:
    for line in infile:
        if line.strip() == "":
            outfile.write("\n")  # preserve blank lines
            continue

        parts = line.strip().split('\t')
        if len(parts) == 2:
            token, label = parts
            if label != "O":
                # Remove anything after #
                prefix, tag = label.split("-", 1)
                base_tag = tag.split("#")[0]
                label = f"{prefix}-{base_tag}"
              # If label contains ROOMS_AMENITIES, replace it with "O"
        if label in labels_to_remove:
                label = "O"
        else:
            # handle malformed lines (e.g., no tab)
            outfile.write(line)

In [6]:
# Start from this code cell if your dataset is already in .txt format
# Also run this code cell if you started with .xml format and ran he code cells above
import pandas as pd


df = pd.read_csv(
    'iob_output_relabelled.txt',
    sep='\t',
    header=None,
    names=['Token', 'Label'],
    keep_default_na=False,
    skip_blank_lines=False
)
df.head()

,Token,Label
0,أنصح,O
1,بالنوم,O
2,وليس,O
3,تناول,O
4,الطعام,O


As you can see, we have successfully loaded our data. The **token** column refers to the current token in question, the **Label** column refers to the label of the current token. There are 3 labels:

- **B-ASPECT**: A token that marks the beginning of an aspect
- **I-ASPECT**: A token that is inside an aspect
- **O**: A token that is not part of any aspect

This data format is called the IOB (Inside, Outside, Beginning) format, and this is how most aspect extraction datasets are stored.


Let's also split our data into a training, validation and test parts while we are at it.

We first make a 80:20 split, and keep 80 percent of the data for training the model.

From the 20 percent we wil divide it in half, and use 10 percent for validation and 10 percent for testing.

In [7]:
from sklearn.model_selection import train_test_split
train_df, val_test_df = train_test_split(df, test_size=0.2, shuffle=False)
val_df, test_df = train_test_split(val_test_df, test_size=0.5, shuffle=False)

train_df.to_csv('data/train.txt', sep='\t', header=False, index=False)
val_df.to_csv('data/val.txt', sep='\t', header=False, index=False)
test_df.to_csv('data/test.txt', sep='\t', header=False, index=False)

#### A.2 Sequence Tagger with Flair-NLP: Loading Modules, Data & Defining Parameters

[Flair-NLP](https://github.com/flairNLP/flair) is a widely used toolkit for training advanced sequence tagger models for various applications like Named Entity Recognition, Aspect Based Sentiment Analysis, etc. The library provides a variety of options to train your models like word embeddings, pre-trained transformers, conditional random fields (CRFs), etc. Please refer to the [documentation of FLAIR](https://flairnlp.github.io/docs/intro) for a more detailed overview of it's capabilities.

In this particular example, we will use one of the options available in FLAIR to train our own Sequence Tagger using a pre-trained Transformer model.

Let's begin by loading the necessary modules of Flair.

In [8]:
from flair.embeddings import TransformerWordEmbeddings, WordEmbeddings, StackedEmbeddings
from flair.models import SequenceTagger
from flair.trainers import ModelTrainer

from flair.data import Corpus
from flair.datasets import ColumnCorpus
from flair.data import Sentence

from pathlib import Path
import numpy as np

/home/ubuntu/venv/lib/python3.10/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


Now let's begin by loading out data into Flair

In [9]:
columns = {0: 'text', 1: 'label'} # This specifies which column is the text and which column is the label. In our case, the text is in the first column and the label is in the second column.
data_folder = Path('data') # This is the folder where your data is stored.
corpus = ColumnCorpus(data_folder, columns,
                              train_file='train.txt',
                              dev_file='val.txt',
                              test_file='test.txt')

print(corpus.train[0].to_tagged_string('label'))


2025-06-25 15:06:30,796 Reading data from data
2025-06-25 15:06:30,798 Train: data/train.txt
2025-06-25 15:06:30,799 Dev: data/val.txt
2025-06-25 15:06:30,800 Test: data/test.txt
Sentence[13]: "أنصح بالنوم وليس تناول الطعام موقع مثالي للإقامة قبل رحلة طيران مبكرة ." → ["موقع"/LOCATION]


Now let's define a few variables for our training. These are some key parameters so some additional information about them is added in the comments. Be sure to read them so you can make appropriate choices for your setting.

In [10]:
model_name = 'aubmindlab/bert-base-arabertv2' #This is the pre-trained model we are going to use. You can change this to any other model from the transformers library. To look for available models, you can visit https://huggingface.co/models
fine_tune = True #With this Arabic model do not set fine-tune to false as your model will at the end not be able to train well. With this model, fine-tuning must be set to true. Fine-tuning results in better performance but requires more computational resources and time.
hidden_size = 256 #This is the size of the hidden layer of the model. You can change this to any other value depending on the computational resources you have. If your training is taking too long, experiment with reducing this number.
use_crf = True #With this model, we need crf to be set true, so that the model can look at the whole sentence and make sure labels follow logical patterns. It trains the model to look at and think about the flow of labels, so how they connect.
output_model_path = 'ar_aspect_extraction_model' #This is the path where the trained model will be saved.

#### A.3 Sequence Tagger with Flair-NLP: Final Setup & Training

Perfect, now let's setup all the parameters and details of our model to-be trained soon!

In [11]:
# 1. Which column do we want to predict?
label_type = 'label'

# 2. Make the label dictionary from the corpus, i.e. a mapping of labels to numbers
label_dict = corpus.make_label_dictionary(label_type=label_type, )
print(label_dict)

# 3. Initialize fine-tuneable transformer embeddings WITH document context. These are the embeddings that will be used for classifying the tokens into aspects.
embeddings = TransformerWordEmbeddings(model = model_name,
                                       layers="-1", #ONLY USE THE LAST LAYER (common practice, but can experiment with other layers)
                                       subtoken_pooling="first",
                                       fine_tune= fine_tune,
                                       use_context=True, #document context is considered during the embedding process (surrounding words, ...)
                                       )

# 4. Initialize our sequence tagger, you can experiment with the hyperparameters here to suit your needs. Some tinkering might help you reach better performance for your dataset.
tagger = SequenceTagger(hidden_size=hidden_size,
                        embeddings=embeddings,
                        tag_dictionary=label_dict,
                        tag_type=label_type,
                        use_crf=use_crf,
                        use_rnn=False,
                        reproject_embeddings=False,
                        )

# 5. Initialize trainer
trainer = ModelTrainer(tagger, corpus)

2025-06-25 15:06:31,964 Computing label dictionary. Progress:


0it [00:00, ?it/s]
3830it [00:00, 37852.77it/s]

2025-06-25 15:06:32,073 Dictionary created for label 'label' with 5 values: HOTEL (seen 1598 times), SERVICE (seen 1341 times), ROOMS (seen 867 times), LOCATION (seen 684 times), FOOD_DRINKS (seen 590 times)
Dictionary with 5 tags: HOTEL, SERVICE, ROOMS, LOCATION, FOOD_DRINKS


2025-06-25 15:06:33,618 SequenceTagger predicts: Dictionary with 21 tags: O, S-HOTEL, B-HOTEL, E-HOTEL, I-HOTEL, S-SERVICE, B-SERVICE, E-SERVICE, I-SERVICE, S-ROOMS, B-ROOMS, E-ROOMS, I-ROOMS, S-LOCATION, B-LOCATION, E-LOCATION, I-LOCATION, S-FOOD_DRINKS, B-FOOD_DRINKS, E-FOOD_DRINKS, I-FOOD_DRINKS


Great, everything seems to be ready, it's time to start training. Remember, this can take quite a while, so grab yourself a cup of coffee in the meantime!

If you have GPU available, switch from CPU to GPU! CPU requires a lot of time and might crash if the training set and model are big!

In [12]:
trainer.fine_tune(output_model_path,
                  learning_rate=5.0e-6, # Can be tuned for your data
                  mini_batch_size=4, # Should be adjusted based on your computation resources. If the code crashes due to memory errors, reduce it. If you have a big GPU, you can increase it to speed up training.
                  mini_batch_chunk_size=1,  # Remove this parameter to speed up computation if you have a big GPU
                  max_epochs=15, # You can adjust the training epochs. The higher the number of epochs is, the better your model trains, yet it takes more time!
                  )

2025-06-25 15:06:33,630 ----------------------------------------------------------------------------------------------------
2025-06-25 15:06:33,631 Model: "SequenceTagger(
  (embeddings): TransformerWordEmbeddings(
    (model): BertModel(
      (embeddings): BertEmbeddings(
        (word_embeddings): Embedding(64001, 768, padding_idx=0)
        (position_embeddings): Embedding(512, 768)
        (token_type_embeddings): Embedding(2, 768)
        (LayerNorm): LayerNorm((768,), eps=1e-12, elementwise_affine=True)
        (dropout): Dropout(p=0.1, inplace=False)
      )
      (encoder): BertEncoder(
        (layer): ModuleList(
          (0-11): 12 x BertLayer(
            (attention): BertAttention(
              (self): BertSdpaSelfAttention(
                (query): Linear(in_features=768, out_features=768, bias=True)
                (key): Linear(in_features=768, out_features=768, bias=True)
                (value): Linear(in_features=768, out_features=768, bias=True)
                

/home/ubuntu/venv/lib/python3.10/site-packages/flair/trainers/trainer.py:545: FutureWarning: `torch.cuda.amp.GradScaler(args...)` is deprecated. Please use `torch.amp.GradScaler('cuda', args...)` instead.
  scaler = torch.cuda.amp.GradScaler(enabled=use_amp and flair.device.type != "cpu")


2025-06-25 15:06:57,493 epoch 1 - iter 95/958 - loss 4.74335731 - time (sec): 23.84 - samples/sec: 343.68 - lr: 0.000000 - momentum: 0.000000
2025-06-25 15:07:20,666 epoch 1 - iter 190/958 - loss 4.50830923 - time (sec): 47.02 - samples/sec: 360.54 - lr: 0.000001 - momentum: 0.000000
2025-06-25 15:07:43,517 epoch 1 - iter 285/958 - loss 4.12774296 - time (sec): 69.87 - samples/sec: 363.25 - lr: 0.000001 - momentum: 0.000000
2025-06-25 15:08:06,566 epoch 1 - iter 380/958 - loss 3.63167764 - time (sec): 92.91 - samples/sec: 368.05 - lr: 0.000001 - momentum: 0.000000
2025-06-25 15:08:30,339 epoch 1 - iter 475/958 - loss 3.17818836 - time (sec): 116.69 - samples/sec: 371.97 - lr: 0.000002 - momentum: 0.000000
2025-06-25 15:08:53,423 epoch 1 - iter 570/958 - loss 2.84376938 - time (sec): 139.77 - samples/sec: 371.45 - lr: 0.000002 - momentum: 0.000000
2025-06-25 15:09:17,229 epoch 1 - iter 665/958 - loss 2.56555691 - time (sec): 163.58 - samples/sec: 374.10 - lr: 0.000002 - momentum: 0.0000

100%|██████████████████████████████████████████████████████████| 30/30 [00:09<00:00,  3.08it/s]

2025-06-25 15:10:40,574 DEV : loss 0.5422455668449402 - f1-score (micro avg)  0.0
2025-06-25 15:10:40,586 ----------------------------------------------------------------------------------------------------


...

2025-06-25 16:08:39,444 DEV : loss 0.19247663021087646 - f1-score (micro avg)  0.6339


2025-06-25 16:08:42,069 ----------------------------------------------------------------------------------------------------
2025-06-25 16:08:42,071 Testing using last state of model ...


100%|██████████████████████████████████████████████████████████| 32/32 [00:09<00:00,  3.26it/s]

2025-06-25 16:08:51,903 
Results:
- F-score (micro) 0.6282
- F-score (macro) 0.6231
- Accuracy 0.4821

By class:
              precision    recall  f1-score   support

       HOTEL     0.6250    0.6114    0.6181       229
     SERVICE     0.6786    0.7268    0.7018       183
       ROOMS     0.5490    0.7568    0.6364       111
    LOCATION     0.4234    0.4747    0.4476        99
 FOOD_DRINKS     0.6974    0.7260    0.7114        73

   micro avg     0.6013    0.6576    0.6282       695
   macro avg     0.5947    0.6591    0.6231       695
weighted avg     0.6059    0.6576    0.6286       695

2025-06-25 16:08:51,903 ----------------------------------------------------------------------------------------------------


{'test_score': 0.6281786941580756}

#### A.4 Testing the Model


Our model is ready, want to test it on a random sentence? Let's do it!

In [13]:
sentence = Sentence('موقع الفندق ممتاز وقريب من جميع الخدمات، لكن مستوى النظافة في الغرف كان غير مرضٍ.')

# predict aspect tags
tagger.predict(sentence)
print(sentence)

Sentence[16]: "موقع الفندق ممتاز وقريب من جميع الخدمات، لكن مستوى النظافة في الغرف كان غير مرضٍ." → ["موقع"/LOCATION, "الغرف"/ROOMS]


As, you can see, even with some conservative settings we are able to have a pretty good aspect extraction model!

To make this model even better, you can try increasing the **hidden_size**.

You can also further tune the hyper-parameters such as **learning_rate** since the optimal values can vary significantly depending on the model and data.


### Task B: Sentiment Classification

Since our model for Task A is capable of identifying interesting aspects in a sentence, we can move on to training a system for the next task of ABSA, ie. Sentiment Classification.
In this task we want to associate a sentiment label (eg: Positive, Negative, Neutral) to each aspect found by our model from Task A.

#### B.1 Loading and checking your Data

Once more, we will begin by first loading our data. The format of this file will look different than the previous file. This time we also need sentiment information for the aspects.

For this task, we need our dataset to be stored in the Comma-seperated Values (CSV) format. Our sample datset is in **.xml format** which is why we use the following code cell to convert it to **.cvs format**. Your data might look different so make sure to adapt this code for your data format.

In [14]:
import xml.etree.ElementTree as ET
import pandas as pd
import csv

xml_path = "AR_Hotels_Train_SB1.xml"
csv_path = "ABSA_sentiment_output.csv"

excluded_categories = {"ROOMS_AMENITIES", "FACILITIES"}

tree = ET.parse(xml_path)
root = tree.getroot()

with open(csv_path, "w", encoding="utf-8", newline='') as csvfile:
    writer = csv.DictWriter(csvfile, fieldnames=["sentence_id", "sentence", "aspect", "category", "polarity"])
    writer.writeheader()

    for review in root.findall("Review"):
        for sentence in review.findall(".//sentence"):
            text_element = sentence.find("text")
            if text_element is None or text_element.text is None:
                continue

            text = text_element.text.strip()
            sentence_id = sentence.attrib.get("id", "")

            opinions = sentence.find("Opinions")
            if opinions is None:
                continue

            for opinion in opinions.findall("Opinion"):
                full_category = opinion.attrib.get("category")
                base_category = full_category.split("#")[0]

                if base_category in excluded_categories:
                    continue  # Skip opinion

                polarity = opinion.attrib.get("polarity")
                aspect = opinion.attrib.get("target", "")

                writer.writerow({
                    "sentence_id": sentence_id,
                    "sentence": text,
                    "aspect": aspect,
                    "category": base_category,
                    "polarity": polarity
                })

df = pd.read_csv('ABSA_sentiment_output.csv')

df.dropna(inplace=True) # Remove rows with missing values
df.head()

,sentence_id,sentence,aspect,category,polarity
0,456:0,أنصح بالنوم وليس تناول الطعام موقع مثالي للإق...,موقع,LOCATION,positive
1,456:1,كانت الغرفة ممتازة وكذلك الموظفون وبوفيه الإفط...,الغرفة,ROOMS,positive
2,456:1,كانت الغرفة ممتازة وكذلك الموظفون وبوفيه الإفط...,الموظفون,SERVICE,positive
3,456:1,كانت الغرفة ممتازة وكذلك الموظفون وبوفيه الإفط...,بوفيه الإفطار,FOOD_DRINKS,positive
4,456:1,كانت الغرفة ممتازة وكذلك الموظفون وبوفيه الإفط...,وجبة العشاء,FOOD_DRINKS,negative


As you can see, the data format looks quite different this time. We have the full text in the **sentence** column. The aspect is stored in the **aspect** column. While the sentiment label we will use is in the **polarity** column. 

Lastly, now we will convert the data to a simpler format, ie. lists for easier processing in the future.

In [15]:
sentence_list = list(df["sentence"])
aspect_list = list(df["aspect"])
tag_list = list(df["polarity"])

raw_data = [[sent, asp, tag] for sent, asp, tag in zip(sentence_list, aspect_list, tag_list)]

#### B.2 Load model & create tagset

First let's define a few variables and load the model we will use.

In [16]:
import torch
from transformers import AutoTokenizer, AutoModel

model_name = 'Walid-Ahmed/arabic-sentiment-model' #This is the pre-trained model we are going to use. You can change this to any other model from the transformers library. To look for available models, you can visit https://huggingface.co/models
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu') # This will automatically use the GPU if it is available, otherwise it will use the CPU.

# Load the pre-trained model and tokenizer
tokenizer = AutoTokenizer.from_pretrained(model_name)
model = AutoModel.from_pretrained(model_name)

We need to assign a number to each sentiment label so the model can interpret it. In Task A this was done using the **corpus.make_label_dictionary** functionality, but that only works when using flair. For this Task, we will be using **transformers**, so we will have to find another way.

In [17]:
tags = list(set(df["polarity"]))
tag2idx = {tag:idx for idx, tag in enumerate(tags)}
idx2tag = {idx:tag for idx, tag in enumerate(tags)}
print(idx2tag)

{0: 'positive', 1: 'neutral', 2: 'negative'}


#### B.3 Helper functions for manipulating data

Now we need to change the format of the data a bit. This is necessary because transformers break down words into smaller units called sub-words. This might cause some confusion, since an aspect can be broken down into multiple parts and only one part gets the annotated sentiment. In that case we also need to adjust our data so we can assign the sentiment label to all parts and not only one. To know more about how tokenization in transformers works and the technical details around it please take a look at [this guide](https://docs.mistral.ai/guides/tokenization/).

In [18]:
#This functions finds the index position of the aspect term in the sentence

def get_pos(sent_list,aspect_list):
    first_pos = sent_list.index(aspect_list[1]) #first position bc [0] is always the CLS token in transformers
    final_pos = []
    for i in range(0,len(aspect_list)-2):
        final_pos.append(first_pos+i)
    return final_pos

In [19]:
#This class will construct a Torch dataset from the tagged sentences. Each instance in the dataset will contain the words, their position in the sentence and tags for those words.

from torch.utils import data
class ABSADataset(data.Dataset):
    def __init__(self, tagged_sents):
        sents, aspects, tags = [], [], [] # list of lists
        bugged = 0
        for sent in tagged_sents:
            try:
                sent_tokens = tokenizer.encode(sent[0])
                aspect_tokens = tokenizer.encode(sent[1])
                pos_aspects = get_pos(sent_tokens, aspect_tokens)
                tag = sent[2]
                sents.append(sent_tokens)
                aspects.append(pos_aspects)
                tags.append(tag)
            except:
                bugged+=1
        print("Ignoring {} Buggy Annotations".format(bugged))
        self.sents, self.aspects, self.tags = sents, aspects, tags

    def __len__(self):
        return len(self.sents)

    def __getitem__(self, idx):
        words, aspects, tags = self.sents[idx], self.aspects[idx], tag2idx[self.tags[idx]] # words, tags: string list
        return words, aspects, tags

#### B.4 Creating our Model for extracting embeddings

Let's wrap our loaded model in a class and then get embeddings for each aspect from the transformer. We will then use these embeddings to classify the sentiment of the aspect.

In [20]:
#This class defines a network where we pass full sentences to a transformer for the entire context but only store the embeddings for the aspect terms.
#This is done by using the position index of the aspect term in the sentence we stored using our previous functions.

import torch
from torch import nn

class Net(nn.Module):
    def __init__(self):
        super().__init__()
        self.device = device
        self.model = AutoModel.from_pretrained(model_name, output_hidden_states=True)

    def forward(self, sent, aspects, y):
        '''
        x: (N, T). int64
        y: (N, T). int64
        '''
        sent = torch.LongTensor(sent).to(self.device)
        aspects = torch.LongTensor(aspects).to(self.device)
        y = torch.LongTensor(y).to(self.device)
        input_ids = sent.unsqueeze(0)  # Batch size 1

        with torch.no_grad():
            outputs = self.model(input_ids)
            last_hidden_states = outputs.last_hidden_state[0]  # Get last hidden state (embeddings)
            start = 0
            end = len(last_hidden_states)-1
            context_window = 5 # define context window we're interested in (5 before and 5 after)

            if aspects[0]-context_window>0:
                start = aspects[0]-context_window
            if aspects[-1]+context_window<len(last_hidden_states)-1:
                end = aspects[-1]+context_window

            all_aspects = []
            for i in range(start,end):
                all_aspects.append(i)

            #For each index in all_aspects, it assigns the corresponding BERT embedding from the last_hidden_states tensor to the corresponding row in the bert_embeds tensor.
            #Each row of bert_embeds now holds the BERT embedding for the corresponding aspect.

            bert_embeds = torch.zeros(len(all_aspects),768).to(self.device)
            for i, aspect in enumerate(all_aspects):
                bert_embeds[i] = last_hidden_states[aspect]

            #  calculates the mean of the embeddings along axis 0 (the rows).
            #  This is done to obtain a single aggregated BERT embedding that represents the information from the context window around the aspects.

            embedding = torch.mean(bert_embeds, axis=0).to(self.device)

        return embedding

In [21]:
#This function will run our network from the previous cell on our entire dataset, therefore extracting and storing embeddings for each aspect term.

def extract(model, iterator):
    model.eval()

    Words, Aspects, Y, Y_hat = [], [], [], [],
    with torch.no_grad():
        for i, batch in enumerate(iterator):
            words, aspects, y = batch

            _, _, y_hat, _ = model(words, aspects, y)  # y_hat: (N, T) = predicted labels
            print(y_hat)

            Words.extend(words)
            Aspects.extend(aspects)
            Y.extend(y.numpy().tolist())
            Y_hat.extend([y_hat.cpu().numpy().tolist()])

    ## calc metric
    print(classification_report(Y, Y_hat))

#### B.4 Initialising the Model & exctracting the embeddings

Now we can begin to use all the massive functions we have defined in the previous sections and initialize first, our dataset, and then our model to be used for extracting the embeddings


In [22]:
from torch.optim import AdamW

dataset = ABSADataset(raw_data)
data_iterator = data.DataLoader(dataset=dataset,
                             batch_size=1,
                             shuffle=False,
                             num_workers=0,
                             pin_memory=False)

Ignoring 1085 Buggy Annotations


Perfect! Now let's initalize our network for extracting the embeddings

In [23]:
model = Net()
model.to(device)

Net(
  (model): BertModel(
    (embeddings): BertEmbeddings(
      (word_embeddings): Embedding(64000, 768, padding_idx=0)
      (position_embeddings): Embedding(512, 768)
      (token_type_embeddings): Embedding(2, 768)
      (LayerNorm): LayerNorm((768,), eps=1e-12, elementwise_affine=True)
      (dropout): Dropout(p=0.1, inplace=False)
    )
    (encoder): BertEncoder(
      (layer): ModuleList(
        (0-11): 12 x BertLayer(
          (attention): BertAttention(
            (self): BertSdpaSelfAttention(
              (query): Linear(in_features=768, out_features=768, bias=True)
              (key): Linear(in_features=768, out_features=768, bias=True)
              (value): Linear(in_features=768, out_features=768, bias=True)
              (dropout): Dropout(p=0.1, inplace=False)
            )
            (output): BertSelfOutput(
              (dense): Linear(in_features=768, out_features=768, bias=True)
              (LayerNorm): LayerNorm((768,), eps=1e-12, elementwise_affine=T

Great, now let's run this model on our entire data to extract embeddings for all the aspects. Again, remember that this can take quite a while, so take a break!

In [24]:
embeddings = []
import tqdm
for i, batch in tqdm.tqdm(enumerate(data_iterator)):
        words, aspects, y = batch
        embedding = model(words, aspects, y)
        embedding = embedding.cpu().numpy()
        y = int(y.cpu().numpy()[0])
        embeddings.append([embedding, y])

6569it [00:47, 137.91it/s]


#### B.5 Setting up a Machine Learning Classifier

Now that our embeddings are extracted. We can train a simple ML classifier to detect sentiment for an embedding. Let's first construct our data in the X (embeddings) and Y (labels) format of sklearn and then split it into a train and test split.

In [25]:
X = [x[0] for x in embeddings]
Y = [x[1] for x in embeddings]

Let's see what a X and Y look like

In [26]:
print("___________ Sample Embedding ___________ ")
print(X[100])
print("___________ Sample Sentiment ___________ ")
print(Y[100])

___________ Sample Embedding ___________ 
[-5.93174212e-02 -1.81604221e-01 -3.63338083e-01 -6.40588328e-02
  6.19046450e-01  4.83593971e-01 -3.74864876e-01  1.50123239e-01
  7.52107203e-02  3.16303931e-02  4.83857632e-01  1.65168017e-01
  3.32096145e-02 -4.12915885e-01  3.71975839e-01  2.77841181e-01
  1.69453591e-01  5.51478490e-02  5.29130697e-01 -2.44303107e-01
 -2.87724454e-02  1.28027752e-01  8.64632964e-01 -6.92802966e-02
  8.31239671e-02  2.06367955e-01 -1.81361586e-01 -3.00804824e-01
  3.64042848e-01 -1.85096994e-01 -5.94020225e-02 -7.95968026e-02
 -2.49010324e-01 -1.32809937e-01  1.15697742e-01 -2.27052659e-01
 -5.83343878e-02 -1.39217302e-01 -8.89187306e-02 -6.06764317e-01
  3.16121101e-01  8.44251364e-02  3.71199667e-01 -4.90562432e-02
  6.68017745e-01  3.86537937e-03 -7.39151716e-01  4.71109927e-01
 -2.36981332e-01  2.26223841e-01 -1.07240953e-01 -2.08087310e-01
  2.56718956e-02 -3.97177637e-02  4.57229204e-02  4.79988128e-01
  2.88102001e-01  4.37300712e-01 -4.30738509e-01

As you can see, X is a very long (768 dimensional) embedding of the aspect, while Y is the ID of the sentiment label (in this case 3)

Let's split X and Y now.

In [27]:
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(X, Y, test_size=.2, shuffle=True, random_state=42, stratify=Y)

#### B.6 Train a ML Classifier on the Embeddings

Finally let's train a simple Linear SVM on the Embeddings. You can of course try to experiment with the classifier you want to use. Depending on the size of the data and the amount of labels, there might be other better options. Some commonly used classifiers include Decision Trees, Multi-layer Perceptron, Random Forests, etc.

In [28]:
from sklearn.svm import SVC
classifier = SVC(kernel="linear", C=0.025)

classifier.fit(X_train, y_train)


,C,0.025
,kernel,'linear'
,degree,3
,gamma,'scale'
,coef0,0.0
,shrinking,True
,probability,False
,tol,0.001
,cache_size,200
,class_weight,None
,verbose,False


Our  model is trained! Let's find it out how accurate it is on the Test set.

In [29]:
from sklearn.metrics import classification_report

predictions = classifier.predict(X_test)
print(classification_report(y_test, predictions, zero_division=0))

              precision    recall  f1-score   support

           0       0.87      0.93      0.90       821
           1       0.00      0.00      0.00        93
           2       0.81      0.89      0.85       400

    accuracy                           0.85      1314
   macro avg       0.56      0.61      0.58      1314
weighted avg       0.79      0.85      0.82      1314



As you can see, our model has an accuracy of 85% with more than 1000 labeled aspects labelled with sentiment used for training!